In [1]:
using LinearAlgebra

using Random 
using PlotlyJS
using ImplicitEquations
using Pkg
# Pkg.add(url="https://github.com/matthiashimmelmann/Implicit3DPlotting.jl.git")
using MultivariatePolynomials, DynamicPolynomials
using ExtXYZ
using HomotopyContinuation
using Clustering

using Symbolics
using JuMP
using Optim
using SymEngine
using RANSAC
using RANSACVisualizer
using Distributions
using Makie

using ForwardDiff

using Distributions
using Roots
using HTTP, CSV, DataFrames
using DelimitedFiles

WebIO._IJuliaInit()

In [2]:
#@axl start
#@axl A = point(0.,0.5,0.)
#@axl view

#WRITE THIS TODO HERE FOR TOMORROW: INVENT YOUR OWN COMPLEX
#GENERATE ONE VARIABLE FOR EACH OF THESE POINTS
#NOW WE HAVE A POLYHEDRAL COMPLEX (SQUARES FOR (1,1))
#FOR EACH MAXIMAL CELL OF THE COMPLEX BUILD A SPHERE

# THE OPTIMAND WILL BE:
# LOOP OVER ALL THE POINT CLOUD
# IF THE POINT BELONGS TO 

In [3]:
data = readdlm("cloud_3.xyz", Float64)

features = collect(data[:, 1:3]')

xdat = zeros(0)
ydat = zeros(0)
zdat = zeros(0)

for i = 1:10
       xdat = vcat(xdat,data[i,1])
       ydat = vcat(ydat,data[i,2])
       zdat = vcat(zdat,data[i,3])
end

layout = Layout(
    title="Pointcloud_initial",
    autosize=true,

)

trace = PlotlyJS.scatter3d(x=xdat, y=ydat, z=zdat, mode = "markers", marker=attr(size=12,colorscale="Viridis", opacity=0.8), marker_size=1.3, marker_sizeref=0.3)



scatter3d with fields marker, mode, type, x, y, and z


In [4]:
dataposition = data[:,1:3] 

datanormals = data[:,4:6]

66134×3 Matrix{Float64}:
 -0.843001  -0.537801   0.010901
 -0.844667  -0.535034   0.016642
 -0.798804  -0.589238  -0.12129
 -0.801501  -0.586671  -0.115813
 -0.804155  -0.584093  -0.110319
 -0.912796  -0.407428   0.028398
 -0.916014  -0.398404   0.046834
 -0.913918  -0.404426   0.034537
 -0.916986  -0.395384   0.052989
 -0.914991  -0.401418   0.040683
 -0.846285  -0.532259   0.022393
 -0.916014  -0.398404   0.046834
 -0.847857  -0.529477   0.028153
  ⋮                    
  0.980604   0.185028   0.064652
  0.979782   0.190549   0.060971
  0.981275   0.182729   0.060901
  0.989921   0.127836   0.060944
  0.978187   0.198058   0.062638
  0.967179   0.246714   0.060802
  0.980457   0.188237   0.057199
  0.981275   0.182729   0.060901
  0.980457   0.188237   0.057199
  0.981929   0.180412   0.057158
  0.968      0.244433   0.056827
  0.960112   0.273558   0.057893

In [5]:
Symbolics.@variables x
Symbolics.@variables y
Symbolics.@variables z
Symbolics.@variables P[1:4,1:3]
Symbolics.@variables M1[1:3,1:3]
Symbolics.@variables M2[1:3,1:3]
Symbolics.@variables M[1:4,1:4]

data = readdlm("cloud_3.xyz", Float64)

center = data[53422,:]

subs1 = Dict(M1[1,1] => x - P[1,1], M1[1,2] => y - P[1,2], M1[1,3] => z - P[1,3], M1[2,1] => x - P[2,1], M1[2,2] => y - P[2,2], M1[2,3] => z - P[2,3], M1[3,1] => x - P[3,1], M1[3,2] => y - P[3,2], M1[3,3] => z - P[3,3])
    
subs2 = Dict(M2[1,1] => x - P[1,1], M2[1,2] => y - P[1,2], M2[1,3] => z - P[1,3], M2[2,1] => x - P[4,1], M2[2,2] => y - P[4,2], M2[2,3] => z - P[4,3], M2[3,1] => x - P[3,1], M2[3,2] => y - P[3,2], M2[3,3] => z - P[3,3])

subs = Dict(M[1,1] => x - P[1,1], M[1,2] => y - P[1,2], M[1,3] => z - P[1,3], M[1,4] => substitute(det(collect(M1)), subs1), M[2,1] => x - P[2,1], M[2,2] => y - P[2,2], M[2,3] => z - P[2,3], M[2,4] => 0, M[3,1] => x - P[3,1], M[3,2] => y - P[3,2], M[3,3] => z - P[3,3], M[3,4] => substitute(det(collect(M2)), subs2), M[4,1] => x - P[4,1], M[4,2] => y - P[4,2], M[4,3] => z - P[4,3], M[4,4] => 0)
    
opt = 0*P[1,1]

c = rand(Uniform(0.,1.))

epsilon = 0.7

usedata = []

for i = 1:size(data)[1]
    if (norm(dataposition[i,:] - center[1:3]) < epsilon)
        subsdata = Dict(x => data[i,1], y => data[i,2], z => data[i,3])
        opt += substitute(substitute(det(collect(M)), subs), subsdata)
        usedata = vcat(usedata, i)
    end
end



opt = 1000*opt


f = eval(build_function(opt^2, P))

println(usedata)


Any[53366, 53371, 53372, 53373, 53374, 53379, 53380, 53383, 53384, 53385, 53386, 53388, 53394, 53395, 53397, 53399, 53400, 53401, 53402, 53403, 53404, 53405, 53408, 53411, 53412, 53414, 53415, 53418, 53419, 53420, 53421, 53422, 53423, 53424, 53429, 53430, 53431, 53432, 53433, 53434, 53435, 53438, 53439, 53440, 53443, 53444, 53445, 53446, 53447, 53449, 53450, 53453, 53454, 53457, 53458, 53460, 53461, 53462, 53466, 53467, 53468, 53471]


In [13]:
usepoints = data[usedata,:]

Ev = [usepoints[1,1] usepoints[1,2] usepoints[1,3] 
    usepoints[10,1] usepoints[2,2] usepoints[2,3]
    usepoints[20,1] usepoints[3,2] usepoints[3,3]
    usepoints[30,1] usepoints[4,2] usepoints[4,3] ]


result = Optim.optimize(f, Ev)

PRes = Optim.minimizer(result)

subsEv = Dict(P[1,1] => Ev[1,1], P[2,1] => Ev[2,1], P[3,1] => Ev[3,1], P[4,1] => Ev[4,1], P[1,2] => Ev[1,2], P[2,2] => Ev[2,2], P[3,2] => Ev[3,2], P[4,2] => Ev[4,2], P[1,3] => Ev[1,3], P[2,3] => Ev[2,3],  P[3,3] => Ev[3,3],  P[4,3] => Ev[4,3])
subsP = Dict(P[1,1] => PRes[1,1], P[2,1] => PRes[2,1], P[3,1] => PRes[3,1], P[4,1] => PRes[4,1], P[1,2] => PRes[1,2], P[2,2] => PRes[2,2], P[3,2] => PRes[3,2], P[4,2] => PRes[4,2], P[1,3] => PRes[1,3], P[2,3] => PRes[2,3],  P[3,3] => PRes[3,3],  P[4,3] => PRes[4,3])
fimplicit = eval(build_function(substitute(substitute(det(collect(M)), subs), subsP), [x, y, z]))


usepoints


62×6 Matrix{Float64}:
 60.0805  70.5069  31.5172  0.108917  0.12016   0.986762
 60.1158  70.9118  31.4685  0.096439  0.10819   0.989442
 60.1277  70.811   31.4766  0.092332  0.104135  0.990268
 60.1333  70.5249  31.5105  0.100553  0.11223   0.988582
 60.1456  70.424   31.5189  0.096439  0.10819   0.989442
 60.2466  70.8726  31.4595  0.100553  0.11223   0.988582
 60.2631  70.4854  31.5013  0.104675  0.116253  0.987688
 60.2977  71.11    31.4293  0.092332  0.104135  0.990268
 60.3162  70.7222  31.4697  0.096439  0.10819   0.989442
 60.3203  70.9378  31.442   0.104675  0.116253  0.987688
 60.3331  70.3347  31.5117  0.100553  0.11223   0.988582
 60.3528  70.172   31.5256  0.113167  0.124051  0.985802
 60.4088  70.9552  31.4337  0.088235  0.100064  0.991061
  ⋮                                             ⋮
 60.9705  70.6875  31.409   0.084147  0.095978  0.99182
 60.9892  70.2975  31.4485  0.088235  0.100064  0.991061
 61.0998  70.6003  31.4036  0.100553  0.11223   0.988582
 61.1153  70.2104

In [14]:
k = 3000


v = rand(Float64,k,3)

A = []

centernormal = datanormals[23444,:]

camera = center[1:3] - 3*centernormal



for i = 1:k
    randomsmall = rand(Uniform(0,epsilon),3)
    h(t) = fimplicit(camera + t*(centernormal +randomsmall))
    roots = find_zeros(h,-1000,1000)
    minimum = 1000
    for j = 1:length(roots)
        if(roots[j] < minimum && roots[j] > 0) minimum = roots[j]
        end
    end
   
    A = vcat(A, [camera + minimum*(centernormal +randomsmall)])
   
end


for i in 1:k
    for j in 1:3
        print(A[i][j])
        print(" ")
    end
    println()
end

60.12458431551116 71.18483526776076 31.544367258367775 
60.52739314462308 70.88442796337837 31.507932882144694 
60.16549293620674 70.94844698695837 31.48513027116903 
59.55544255752363 70.36362996489267 31.254382231562516 
59.76829333314879 70.60117272441465 31.34395599251155 
59.94406773376732 71.5172347088623 31.613049305474064 
59.4553512494857 70.08679411337074 31.167515313562664 
60.12346084102064 70.30809400589288 31.30650659976915 
60.99617615476271 70.55630350457656 31.00699823400685 
60.915773901694955 71.65958756497277 31.485033614542765 
60.16033248115388 70.64872479460065 31.403244474541733 
60.69536066966099 71.2849882060527 31.522215413097058 
59.357180656891146 70.1987192787636 31.186223248395294 
60.0272627381931 71.46816685784636 31.609619813707535 
59.2639337591651 70.56685818940309 31.274933840043833 
59.93047420955002 71.62893872108732 31.641709356184908 
59.84798488202785 70.54498759153891 31.33816235593565 
60.89350826023749 70.36495817475064 31.017914277031625 
5

60.69213926567934 71.68872082649783 31.676698665480167 
61.0640983136979 70.65186741555443 30.988147860679604 
60.888453915588556 71.53433815056566 31.46009541096989 
60.74272440474638 71.07240275194499 31.40429232430929 
60.82214436101204 71.86772079652042 31.638561150603703 
60.09831387132031 70.17791489084941 31.268233143133873 
61.31328529301034 71.89184898864981 31.2520423053562 
59.844559369393934 71.52735943019404 31.60399293942432 
61.295216346222624 71.68325406833029 31.188459584489646 
60.839776140285174 71.1563855012451 31.35764262471245 
60.603087040140586 71.08273839454075 31.52080088424695 
61.08361244853883 71.98230794655944 31.47080117768799 
60.06176957459663 70.79355437563814 31.430849082387052 
59.85409817747445 70.7150572814766 31.384980942089165 
61.2275226121969 71.52692145410361 31.18438513532811 
60.38727098763343 70.66589397992585 31.434662293558482 
59.457222497714426 70.58181504994839 31.301876449647903 
59.509230942100196 70.56858067344356 31.304448968887357

59.76632300168429 70.92665719714529 31.43193207639768 
59.94235993497965 70.4421148766444 31.32144577594836 
60.44939811047636 71.12575543908291 31.56702994021868 
59.69431311875375 70.54529275614588 31.3200530242535 
60.64823620104666 71.43691470713476 31.617616581304627 
60.40330664351449 70.92609724499907 31.507274091520316 
59.94402222518421 70.4417872527113 31.321553604010752 
60.675860148836605 70.61515581185903 31.286671446543203 
61.29475717310144 72.42733428512977 31.467521798170303 
61.31668268197039 72.71128313102297 31.55623197105624 
59.85543252939008 70.41772881841314 31.304551030614842 
59.9447291120472 70.78771288827926 31.415405820332165 
60.01783108199898 71.60669415919398 31.646044982164813 
60.68645513068803 72.12331602659185 31.855915831577672 
60.664989606299244 71.00944232486675 31.443276694135914 
59.61836716582453 70.94135335510217 31.41838820270295 
60.704303882423616 72.22022898676907 31.87034914224234 
61.42570931666404 73.06132448393923 31.599571207405365 


59.84212188075798 70.85026749678603 31.42020808235128 
60.10182787227331 70.31292110710105 31.30526355023285 
60.85258227531977 71.50251699513898 31.47706132874695 
60.023258607987145 70.85431505075009 31.44275970632377 
60.370895348072565 71.04107115578022 31.534632657967553 
59.54330918576683 71.01841079070418 31.430377564589772 
61.25958719138781 73.06414305877882 31.734400255647042 
61.11342844731544 72.52536995810847 31.65026572847333 
60.55632455112619 70.70411179760397 31.415986390965493 
60.63370819697998 70.32764923462716 31.21277943222261 
60.65812355105543 71.54748481332807 31.651290048984762 
60.90709993925678 71.28085054313756 31.350090326394973 
60.71528236281066 71.0684478020027 31.424905889456948 
59.8233050787321 70.76862396993126 31.395852406174644 
59.24972831751768 70.46021039242804 31.244354731282346 
59.80661685533763 71.11851421104015 31.488699640564658 
60.82956737106293 70.9404714995511 31.284949791209563 
60.51670696746732 70.65931540570571 31.430343380816037 

59.36810644955578 70.95336762099427 31.39199917188673 
60.02994228338017 70.25676380271626 31.28154827683332 
60.36555899537316 71.3779984740383 31.62540367474555 
61.35620015559142 72.20429179730603 31.334522773719026 
59.78297073961354 70.19046384349087 31.23437912086228 
60.968416261527814 70.82677673737312 31.130638816264916 
60.11533663223207 70.54720248662183 31.37039335413103 
60.38530431583219 70.64502052499718 31.428755690416562 
59.76519696802851 70.1447117768737 31.21987603597815 
61.01496910726846 70.86836154099214 31.10876027739551 
59.4308424287919 70.22586664008416 31.202300032100304 
59.33724277720091 70.2413295306232 31.19540880425608 
59.71202402384676 71.05104181469788 31.45920720372508 
59.93230279464095 70.28465857985059 31.27757250985429 
59.97838516611784 71.04090840123793 31.488021169311715 
59.859160972087395 70.70966729724879 31.384119571949828 
60.75315059736083 70.48388069760756 31.175339435984057 
60.9259906924985 70.9100771817975 31.195977347425718 
59.133

...

60.42242969942992 71.82447168990916 31.753305959362176 
59.73950260756783 70.37960492954737 31.28049885885425 
60.720265264906736 71.20808435672573 31.473267513870656 
60.97424687590866 71.02053247567436 31.198525800441153 
60.66386499850769 71.70588870116903 31.706648602266423 
59.24021682411572 70.3375540616617 31.209993841592183 
60.949681532874436 71.54552266970506 31.414983233328037 
59.85217982014329 70.75604816894499 31.395863520995967 
60.35803294983502 70.99011916823873 31.51927400778018 
60.58312369145923 71.23770530531634 31.596667842456224 
59.48887475646336 70.30781265206228 31.231375882777126 
60.979616788647164 71.29795927213343 31.298132309026652 
60.003676041329626 70.34510439244511 31.302397037144008 
60.630160081290825 70.39109773519579 31.23940551819212 
60.25524033849823 70.75223793990236 31.442545243516385 
60.49588671170164 70.55299637213642 31.405789869070766 
59.97228977293698 71.26396948791005 31.5477564108483 
59.976696587226144 70.87209138553862 31.442063117

60.441456074889736 71.22024883725793 31.591723768703275 
60.85625323741043 70.44674305124477 31.07851256999915 
59.94181646103636 70.74641257747054 31.403866322655595 
59.801584624385924 70.759433918681 31.390789462561543 
61.23059538157416 72.17029898523973 31.422892962736032 
60.1297168274278 70.26842696229446 31.296484995646487 
61.07528380510932 70.91454704418919 31.07753078674088 
60.92793318538677 71.18819438642134 31.29860610965684 
60.1794993152595 70.88456366871443 31.469467756497192 
60.05894645545547 70.96577933404238 31.47720311100808 
60.845077606671424 71.10861602521024 31.335474289801873 
60.831658849737664 71.28575087972385 31.41266124959161 
60.77165565568266 71.75556789998849 31.63726085704127 
60.30125273085449 71.14873020693969 31.555553670972568 
61.2028439393125 71.11051630268216 31.048289908697637 
61.00900154661845 71.59433168179848 31.38551358012687 
60.71687593790069 72.03747130863864 31.788197053023598 
61.04844637002369 71.58601776882837 31.350646349618277 


60.6409922692082 70.54701513941781 31.2891479376375 
61.161740128379286 72.08434936807642 31.446127135825552 
60.13785528392379 70.71029405888687 31.417281706940607 
60.13170484864179 70.36688675105123 31.323425406245335 
60.94400666380183 70.51401102612147 31.03312450793365 
60.01600549499642 71.29466765206452 31.561260311804556 
59.716136958774605 70.54546744510685 31.322684164584064 
60.29377034390447 70.68301337366552 31.428314347143917 
59.819295497638386 71.04391332540773 31.469984716350965 
60.037638313284184 70.49645610847563 31.347446918834844 
61.225057040851084 71.64214379705565 31.229524434871532 
60.30787369834673 70.86553458778592 31.479514229741053 
59.73132558613068 70.67329411343522 31.35912455270607 
61.06964889854202 72.32862077315895 31.611807359586532 
59.55489811004721 70.50893486155255 31.29369351382367 
59.61932909745592 71.18057854032519 31.483326887418425 
60.90686244558959 71.00502940035221 31.246940293867677 
59.835698663233806 70.18745950029124 31.239801277

59.404210847475994 70.43798779735128 31.256626144276456 
60.84537233359413 71.50762613764027 31.48478478470028 
60.24061143197438 70.30000266300897 31.31809617986409 
60.820381957115195 70.89124197801362 31.27389428419578 
61.186998577585086 71.7600040314732 31.304299600373785 
60.208290031549076 70.71968454418045 31.42816152922734 
61.09505172254874 70.93314484191411 31.068591225195426 
60.72096175307669 70.89801885632035 31.356431154005357 
61.0801529089986 71.56185020096176 31.31607187712014 
60.67354613691985 70.81915646570556 31.365011853260565 
60.87676326736928 70.5324136677336 31.094107107074198 
60.89861457739598 70.60180718876302 31.102524138234042 
60.60260678481392 71.27817218883537 31.595222186110174 
59.57677686348818 70.55773692886575 31.30950876400603 
60.75072022245906 70.64456330913075 31.237500102698988 
60.547811965135566 70.96473765045032 31.521441388567528 
60.376610571460226 71.07148971810335 31.543569116722356 
60.620163698425905 71.1643115493996 31.537678750000

59.975307125661736 70.37628499147175 31.307496634802035 
60.84047933342213 71.11164186729077 31.340309184130994 
60.38255304113386 70.48114240761694 31.38383467816801 
59.75828053951378 71.14046693824568 31.488921274680074 
60.21162816928878 71.12364125969842 31.538104485972887 
60.00178303912885 70.76283364254766 31.415418070839014 
59.3300315291484 70.29715643888447 31.209682229263585 
61.067659175613805 72.41605415274194 31.646175596391984 
61.09933954900931 72.17499835546586 31.530331031685556 
60.79953896096475 71.13297282197618 31.3812582314371 
60.7746627708995 71.07157288070644 31.378269248932522 
60.77674994640657 71.25173779970041 31.44413013689521 
60.40409283742909 70.63398252836616 31.4279361564392 
61.17724877738065 71.54116748614753 31.230179300917744 
59.67442381540062 70.86691627151947 31.404856841284765 
60.47971892185277 70.74011695687844 31.465459697468596 
59.980095790264464 70.8128507161242 31.426407960009286 
60.11429684396034 70.46470829750521 31.347899787375265

In [15]:
for i in 1:length(usepoints)
    for j in 1:3
        print(usepoints[i,j])
        print(" ")
    end
    println()
end

60.080532 70.506889 31.517185 
60.115841 70.911804 31.468452 
60.127731 70.81105 31.476572 
60.133259 70.524872 31.510469 
60.145615 70.423988 31.51889 
60.246586 70.872612 31.459465 
60.263138 70.48542 31.501266 
60.297676 71.109955 31.429295 
60.316238 70.722183 31.469654 
60.320339 70.937813 31.441957 
60.333111 70.334679 31.511734 
60.35284 70.172012 31.52562 
60.408798 70.955238 31.433739 
60.444817 70.179131 31.5165 
60.478897 71.309464 31.391731 
60.498619 70.920914 31.430439 
60.503387 71.09491 31.408108 
60.51664 70.532562 31.470856 
60.520382 70.706741 31.447954 
60.532963 70.144478 31.512999 
60.536331 70.954491 31.4228 
60.536846 70.323257 31.48918 
60.553696 70.566132 31.46302 
60.569366 70.178062 31.50499 
60.582447 71.281776 31.384674 
60.589188 71.026039 31.408577 
60.608891 70.637192 31.448036 
60.686031 71.253403 31.375525 
60.702354 71.109489 31.392029 
60.704079 70.864395 31.413864 
60.720425 70.47567 31.453951 
60.721497 70.720284 31.430756 
60.73624 70.091629 31.4

LoadError: BoundsError: attempt to access 62×6 Matrix{Float64} at index [63, 1]

In [16]:
println(camera)

[58.249089, 69.752796, 30.033914]


In [17]:
println(Ev)

[60.080532 70.506889 31.517185; 60.320339 70.911804 31.468452; 60.532963 70.81105 31.476572; 60.704079 70.524872 31.510469]


In [18]:
println(PRes)

[60.532976400022456 70.89250297258575 31.51306285812802; 60.52991107402884 70.86813457183496 31.506406763783417; 60.58008458701404 71.02420180567475 31.55431590394898; 60.5660991259556 70.9024305581947 31.519982851855247]


In [19]:
fimplicitEv = eval(build_function(substitute(substitute(det(collect(M)), subs), subsEv), [x, y, z]))

k = 3000


v = rand(Float64,k,3)

A = []

centernormal = datanormals[23444,:]

camera = center[1:3] - 1*centernormal



for i = 1:k
    randomsmall = rand(Uniform(0,epsilon),3)
    h(t) = fimplicitEv(camera + t*(centernormal +randomsmall))
    roots = find_zeros(h,-1000,1000)
    minimum = 1000
    for j = 1:length(roots)
        if(roots[j] < minimum && roots[j] > 0) minimum = roots[j]
        end
    end
   
    A = vcat(A, [camera + minimum*(centernormal +randomsmall)])
   
end


for i in 1:k
    for j in 1:3
        print(A[i][j])
        print(" ")
    end
    println()
end


60.68673987624427 70.80619838418292 31.47591554036479 
60.73355512810652 70.69856271943814 31.488588595000756 
60.33400199621356 70.68522155269157 31.49339226624585 
60.61173351072093 70.86132178452972 31.469828161515885 
61.1111141607299 70.81924579637733 31.470327257012762 
60.46302720918235 70.8274392171208 31.475024318222623 
60.466280697581325 70.76041754962709 31.483278379736056 
886.1090565321957 455.59819526827874 855.4575831874708 
1035.4010379080617 429.769819218734 1154.3211337701075 
60.609854474921065 70.76513972938099 31.481599171556432 
60.383780178049946 70.8512269645366 31.47253624909734 
61.269284568940996 70.9338865541704 31.455042289177054 
60.61780074537179 70.74405320538526 31.484118650138377 
911.8875968209505 483.3040756845994 1097.6481861925827 
60.562686454361916 71.07477987900091 31.44424703725731 
60.62429595384356 70.902096372276 31.464766519746938 
60.49117366477255 71.09546824924476 31.442191986247877 
60.43046585880078 70.83208147955405 31.47464323124941

1345.3737319756337 682.0671088189682 1195.1109303187097 
60.789526333068316 70.95975669821273 31.456413011868992 
60.53992345650222 70.7557408325536 31.483332585233402 
61.06618907186228 70.76395857940838 31.477408125327763 
60.57014043157538 70.76803959312032 31.48157442567445 
61.148788018255786 70.96702496907895 31.452232075171963 
60.459383969734795 70.78513520239262 31.480258636002297 
60.58658418642617 70.68406562021983 31.491838200837307 
60.538971263805664 70.87999627230369 31.468090555989086 
60.60477715352959 70.8486372527041 31.471429399600026 
60.50371443644553 70.80641787859058 31.477346752333652 
60.75330499996292 70.83497462205383 31.471828290641877 
1201.863626615142 431.2107767811483 1095.1665950771387 
60.72443392786709 70.94427454794126 31.458839341454116 
60.394913174921186 70.97168937373156 31.45776787719932 
1000.1009483981273 423.6855500957689 796.4560169346682 
60.99971979719713 71.12808068682229 31.43428390880062 
60.94037743598046 70.70377035702367 31.48589891

61.27041437957027 70.79607908941429 31.471536957646663 
60.66987936538819 70.61286215462479 31.4998090196826 
61.025850866526355 71.05082664943636 31.44332731180723 
60.525754156732404 70.76769919113744 31.481962578521365 
60.62607299725916 70.67519616682573 31.49254020338842 
60.72731031184374 70.87645958234083 31.46702945294181 
61.25912454600041 70.80761340792795 31.470266919508283 
60.90382451767685 70.94697417560873 31.45693680041896 
60.98367942922863 71.26777586667669 31.41764375829106 
60.60145844056264 71.17686926347021 31.431625457673658 
60.634734317512276 70.88864793936735 31.46631991598652 
60.71853187602674 70.81389745295287 31.474697309055667 
60.590940155099176 70.88915534443687 31.466596767930028 
60.61471455190989 70.78086312007788 31.479630624816608 
60.76850954263285 71.10389081201028 31.43918767372601 
60.695559417294476 70.96604699523995 31.456440499703803 
60.951627503186124 70.69950484945176 31.48630072920814 
60.41274774454879 70.86533887928373 31.4706597191112

60.82903704686613 70.87077745501509 31.466810019951403 
60.7443521876764 70.97172727354636 31.455351078656623 
1297.9189560883742 430.30762870229864 1162.9061023881945 
1206.8614780927237 516.3571843806945 1181.0864829617778 
60.598527984219096 70.88671475463605 31.46683667239413 
60.46677453944898 70.70702667416336 31.48998892287899 
60.78457604742242 70.7604984556982 31.480566820994866 
60.99837633015367 70.77238407821483 31.477061666805035 
60.57076232916355 71.12844848865741 31.43769155050637 
60.81718560188212 71.17399392086868 31.430336923859155 
60.876237249325285 70.94936078637811 31.456899671999455 
60.466093347990665 70.71282735713444 31.48925168557 
60.517063579948434 70.8063296281521 31.477265238860554 
60.65556907361765 71.24998986375878 31.422413259565012 
60.85511930774469 70.72515578844013 31.484162372383587 
60.943324704150044 71.17871811857904 31.42869755792651 
60.818558760761675 70.6875115892433 31.489083132906043 
60.831236442821535 70.77185603752802 31.47874614059

60.69141438698697 70.79791817984004 31.47688189742184 
60.89702653832438 70.86628332845224 31.466723694364333 
60.82417183436461 70.74315099240307 31.482287601931677 
60.82177461639211 70.83811509878053 31.47082342958401 
60.943889749018744 71.0064548740564 31.449411132812354 
60.779072173136306 71.05253412978152 31.445298825771452 
60.51489707514934 70.84492916879259 31.472544244084148 
60.80068771871763 71.0009318697856 31.451344855672165 
60.364211858769835 70.7903535724683 31.480113412840883 
60.85610864728993 71.04773483658902 31.445221666743002 
60.42482807604131 70.71174430402056 31.48964444705322 
60.58390177300838 70.91218390188253 31.463844896607327 
60.559063396015915 71.08016219714482 31.443618794967996 
60.58483813044956 70.66153897793622 31.494703666002597 
60.822960089285445 70.7363693116253 31.48312040440462 
60.50019935160401 70.81031439728277 31.47689133946946 
60.67576154874319 70.95849738690973 31.457514249137983 
60.71244285789129 70.82694972893225 31.4731661491795

...

60.58254533438841 70.80656795811268 31.476745870918773 
60.53296994232874 70.73329417638915 31.48617227754778 
60.870302292315195 71.13367658738036 31.434747466188306 
60.615730137786905 70.78388586747865 31.47925194351518 
60.40604085950591 70.88608312434073 31.46815591502676 
60.64220182726982 70.92937623974841 31.46131207442291 
1034.4212991603526 465.1920489659469 970.9958758588108 
60.65642113394793 70.90242839350638 31.464471725842806 
60.72532382821733 71.00728533274925 31.451208238355743 
60.68839285484263 70.9113547892918 31.463127557689738 
60.759424754230494 70.76196017129166 31.480627999822968 
60.91728471089337 71.24430643593806 31.421035825624934 
60.62563764649145 70.92400200614102 31.462093425166845 
61.00548004135303 70.72065222227027 31.483216708572165 
942.9320766146799 419.45226447325734 920.6500855079679 
60.379561297759246 70.89514925400756 31.46718067158979 
60.68772657373061 70.79398303371212 31.477393738318593 
60.692344711961894 71.15216695041684 31.4339569492

60.444333101823354 70.79508461194816 31.479119657864597 
60.53892842387151 70.92258397643734 31.462897023038636 
60.49228945516129 70.77658747319015 31.481102653533526 
61.34417561264375 71.01224695712048 31.444935157046444 
60.99886110857517 70.81920657516433 31.47142360165052 
60.39391362592682 70.8278201344379 31.475359434243106 
60.63321670655271 70.64204092507217 31.49660015329394 
60.5422994141678 70.91253014163524 31.464099015140796 
60.433042486569406 70.81122990682603 31.477193601528892 
60.72023590868717 70.68444908197507 31.49044389684254 
60.690211971834046 70.89983463269382 31.464509880706323 
60.722521705809505 70.88071054248722 31.46655550540127 
60.98509231082947 70.69047040816392 31.487053850741077 
61.36835551885208 71.02298718651825 31.44341591493854 
884.275409675 410.3500585082196 1070.2321672733224 
60.741093490364264 70.92716248658024 31.46077006307173 
60.844029423886965 71.02240470937785 31.448379845035472 
61.10676735965187 70.86553281320717 31.464813350026677

60.78765441274403 70.90937970238194 31.462517035137694 
1093.160045831426 536.1235497277579 1056.7581001899055 
60.621151917911675 71.19532624709653 31.42925835595317 
60.5195565402301 70.80428986888046 31.477498435324872 
60.72830904576786 70.88111520893649 31.46645653526455 
60.370105230054705 70.85951028544007 31.471586534712838 
60.44784886233055 70.83989090598867 31.47358544405115 
61.00932006116921 71.19176572626368 31.42654790300009 
60.81246302163414 70.79924159458241 31.475611163433125 
61.36337633219888 70.79187308417733 31.471118617172436 
60.350316810127 70.83032121523105 31.47525592523235 
60.9239058897177 70.66881999925677 31.490280611486796 
60.78856844952677 70.82492509386097 31.472724878807803 
884.4826994485472 476.0548682315968 992.4128962369817 
1295.421081068744 615.7096368843331 1131.770128044815 
60.31797221229279 70.66062698181472 31.49671100948998 
939.3730559345946 461.8330903335016 1059.679136927721 
1192.9190118386366 558.7577072569168 1157.2781237878266 
60

60.95796839314187 70.665697176869 31.490312221305 
1009.3110088679842 620.2103699463378 1147.3769807944152 
60.84150960761134 70.70008867092766 31.487329965126627 
60.47546262778838 71.03597684137699 31.449505699613606 
60.9510956489196 70.65797908429832 31.491312514443838 
60.4460647103892 70.89034883867993 31.46741573230088 
60.595169153475794 70.69045074793418 31.49095441290649 
61.23186626132329 71.19080801965444 31.424626270952757 
1190.0468499273359 602.0575777705149 1070.0888348670353 
60.60999884766978 70.81938808346972 31.474958061474528 
1047.5478198459057 430.5171118619066 1120.6736191876096 
955.0572735172357 400.6951825821975 1180.948472331529 
60.91633293295102 71.3256108138398 31.411268945211674 
60.657961291249194 70.98418563224807 31.454540759998263 
61.25922079737346 71.21468137676877 31.42151111185561 
60.79597093162784 70.84300507467142 31.470469165941648 
935.6143709173637 489.4497307341642 1005.4056699917727 
60.52650483378676 70.91290835765051 31.46416026636699 


60.799137273874706 70.80654653522932 31.47485175576914 
60.57133736528264 70.83972118057856 31.472776068980174 
60.475532007438765 70.99219624368075 31.45482679049135 
1326.2234326699559 477.9001502200875 1073.3552777581926 
60.431907364313645 70.79917154392022 31.478686301883297 
60.712485490398734 70.94936801310706 31.458321957064232 
60.51482081424161 70.86955555160716 31.469531252906346 
60.45408158550857 70.91568064195108 31.464275129456308 
1268.590263405371 569.9550557301235 1097.4356999822767 
60.80109349641016 70.84287237134811 31.47043839557916 
60.68737808270345 70.84819703191013 31.47080294966658 
61.14278965807853 70.76622275902973 31.47637936263718 
60.89626780863601 70.78205319303481 31.476889618007853 
60.61214781878512 70.994263682384 31.453664445893743 
60.49772757648301 70.82295892332908 31.475354032411335 
60.56655000583011 70.76167700405874 31.482387145196967 
60.567231428421174 70.74132300403309 31.48489380428732 
60.854170582045846 70.89205624264522 31.4640113401